# Introduction 
This notebooks shows a basic example of how events and the status byte work.



In [1]:
import visa_mock as pyvisa_mock

from pyvisa import ResourceManager

from visa_mock.base.register import register_resource

from geiger_counter_mock import GeigerCounterMock

gc_address = 'MOCK0::mock1::INSTR'
register_resource(gc_address, GeigerCounterMock())

In [2]:
rm = ResourceManager('@mock')
rm.list_resources()
inst = rm.open_resource(gc_address)
print(inst.query('*IDN?'))

geiger counter:fake corp:123456
